# Practical Session - Final

Student(s):

Aubery Cléa
Shinawi Aymeric

## 1. Problem selection

Choose one of the two following problems (delete the other cell).
You can find extra information regarding both problems and their associated models in
the main course.

### Problem 1: Single-Machine Scheduling

A machine needs to produce a set L of n items. Each
item i has a manufacturing time pi, a release time ri (time at which its components are
available) and a deadline di (time at which the time should be manufactured). The
factory can only produce one item at a time, and manufacturing process cannot be
interrupted (non-preemptive machine), what is the schedule that minimize the
overdue deliveries?

**Variables:**

- $x_{ij}$ - Binary variable indicating if $i$ is manufactured before $j$.
- $s_{i}$ - Continuous variable indicating the starting time of task $i$.
- $y_{i}$ - Binary variable indicating if the item $i$ is overdue.

**Model:**

$$
    \begin{align}
      \text{min.} \quad & \sum_{j=1}^{n} y_{i}                 &                                           \\
      \text{s.t.} \quad & s_j \geq s_i + p_i - M^1_{ij} (1 - x_{ij}), & \forall i,j \in L,\ i \ne j \\
                        & s_i + p_i \leq d_i + M^2_{i} y_i,          & \forall i \in L             \\
                        & x_{ij} + x_{ji} = 1,                 & \forall i,j \in L,\ i \ne j \\
                        & x_{ij} \in \left\{0,~1\right\},      & \forall i,j \in L                         \\
                        & y_{i} \in \left\{0,~1\right\},       & \forall i \in L                           \\
                        & s_{i} \geq r_{i},                    & \forall i \in L
    \end{align}
$$

## 2. Preparation

### 2.1. Instance

Implement a class to hold the various data parameters required for the problem (e.g.,
manufacturing time or aircraft target landing time).
Your class can be a simple data holder with various fields or can contain extra methods
for later use.

In [1]:
def createItem(pi, ri, di):
    return [pi, ri, di]

class Instance:
    def __init__(self):
      self.items=[]

    def addItems(self, items):
      self.items.extend(items)

    def addItem(self, item):
      self.items.append(item)

    def getItems(self):
      return self.items

    def getNbItems(self):
      return len(self.items)

    def getItem(self, i):
      return self.items[i]

    def getPi(self, i):
      return self.items[i][0]

    def getRi(self, i):
      return self.items[i][1]

    def getDi(self, i):
      return self.items[i][2]

In [8]:
# (Test)

# items = [[5, 2, 10], [7, 1, 15], [5, 3, 22], [1, 4, 8]]
print(createItem(5, 2, 10))

instance = Instance()

instance.addItem(createItem(5, 2, 10))
instance.addItem(createItem(7, 1, 15))
print(instance.getItems())

instance.addItems([createItem(5, 3, 22), createItem(1, 4, 8)])
print(instance.getItems())

print(instance.getItem(1))

[5, 2, 10]
[[5, 2, 10], [7, 1, 15]]
[[5, 2, 10], [7, 1, 15], [5, 3, 22], [1, 4, 8]]
[7, 1, 15]


### 2.2. Solution

Implement a to hold a proper solution for the chosen problem.
Your class should not contain the values for all variables in your model but only the
relevant ones.

In [10]:
# need to stock  ?
#    xij - Binary variable indicating if i is manufactured before j.  --> not really, can be deducted
#    yi - Binary variable indicating if the item i is overdue. --> not really, can be deducted
#    si - Continuous variable indicating the starting time of task i.  --> yes
#         How do we know i / the length of s ?
#         We have instance but not initialized.

class Solution:
    instance: Instance
    objective: int
    s:[]
    def __init__(self):
      self.s=[]
      self.objective=-1

    def setInstance(self, instance):
      self.instance = instance
      self.s=[-1 for i in range(instance.getNbItems())]

    def setS(self, S):
      if(len(S)==len(self.s)):
        self.s = S
      else:
        raise Exception("Incorrect length for Y!")

    def setSi(self, i,  si):
      self.s[i]=si

    def getS(self):
      return self.s

    def getSi(self, i):
      return self.s[i]

    def getInstance(self):
      return self.instance

    def setObjective(self, obj):
      self.objective = obj
    
    def getObjective(self):
      return self.objective

Implement the function below to check if a solution is valid or not.

In [11]:

def check_solution(solution: Solution) -> bool:
    instance = solution.getInstance()
    res = True
    for i in range(instance.getNbItems()):
        # item  : ri, pi, di :  release time ri, manufacturing time pi and a deadline di
        # verification : task is executed
        soli = solution.getSi(i)
        pi = instance.getPi(i) 
        ri = instance.getRi(i) 
            
        if(soli==-1):
            return False
        # verification : task is started after release time
        elif(ri>soli):
            return False   
    sol = []
    
    # verification : task is started after previous is ended 
    for index, si in enumerate(solution.getS()):
        sol.append([si, index])
    sol.sort()

    for i in range(instance.getNbItems()-1):
        si_0 = sol[i][0]
        pi_0 = instance.getPi(sol[i][1])
        si_1 = sol[i+1][0]
        if(si_0+pi_0>si_1):
            return False
            
    return True

def check_solution_ontime(solution : Solution) -> bool :
    instance = solution.getInstance()
    res = check_solution(solution)
    for i in range(instance.getNbItems()):
        soli = solution.getSi(i)
        pi = instance.getPi(i)
        di = instance.getDi(i)
        # delay (exec + process > deadline)    
        # possible : return pair of boolean : isValid, isLate
        if pi + soli > (di):
            return False
    return True
    

In [12]:
# (Test)
sol = Solution()

sol.setInstance(instance)
print(sol.getInstance().getItems())
print(sol.getS(), "\n")

sol.setS([1, 8, 15, 7])

# items = [[5, 2, 10], [7, 1, 15], [5, 3, 22], [1, 4, 8]]
# false sol
print(sol.getS())
print(check_solution(sol))

# false sol = 2, 8, 15, 7
sol.setSi(0, -1)
print(sol.getS())
print(check_solution(sol))


# true sol = 2, 8, 15, 7
sol.setSi(0, 2)
print(sol.getS())
print(check_solution(sol))
print(check_solution_ontime(sol))

# true sol (late) = 2, 8, 15, 7
sol.setSi(2, 20)
print(sol.getS())
print(check_solution(sol))
print(check_solution_ontime(sol))


[[5, 2, 10], [7, 1, 15], [5, 3, 22], [1, 4, 8]]
[-1, -1, -1, -1] 

[1, 8, 15, 7]
False
[-1, 8, 15, 7]
False
[2, 8, 15, 7]
True
True
[2, 8, 20, 7]
True
False


### 2.3. Data Generation

Propose, explain and implement a method to generate instances of various sizes (e.g.,
number of items or number of aircraft).

In [13]:
import random

def generate_instance(size: int, rng: random.Random = random.Random(), maxDuration: int = 10) -> Instance:
    # Generation of the n items. [pi, ri, di]
    # pi : On peut choisir une durée max des tâches en paramètre (défaut : 15)
    # ri : On choisi de façon aléatoire entre 0 et (pi-1)*size
    # di : On choisi de façon aléatoire entre pi et pi*size
    items = [[] for i in range(size)]
    for i in range(size):
        ri = int (rng.random()*size*maxDuration)
        pi = int (rng.random()*maxDuration+1)

        di = int(rng.random()*maxDuration) + pi +ri
        items[i] = [pi, ri, di]

    instance = Instance()
    instance.addItems(items)
    return instance


In [17]:
# (Test)

print(type(generate_instance(5)))
print(generate_instance(5).getItems())

<class '__main__.Instance'>
[[10, 48, 67], [2, 5, 16], [8, 21, 31], [9, 2, 20], [1, 3, 4]]


## 3. Model implementation

### 3.1. Model

Implement the function below that should create an appropriate `docplex` model
given an instance of the chosen problem.

Use appropriate values for the big-$M$ constants in the model.

### Problem 1: Single-Machine Scheduling

A machine needs to produce a set L of n items. Each
item i has a manufacturing time pi, a release time ri (time at which its components are
available) and a deadline di (time at which the time should be manufactured). The
factory can only produce one item at a time, and manufacturing process cannot be
interrupted (non-preemptive machine), what is the schedule that minimize the
overdue deliveries?

**Variables:**

- $x_{ij}$ - Binary variable indicating if $i$ is manufactured before $j$.
- $s_{i}$ - Continuous variable indicating the starting time of task $i$.
- $y_{i}$ - Binary variable indicating if the item $i$ is overdue.

**Model:**

$$
    \begin{align}
      \text{min.} \quad & \sum_{j=1}^{n} y_{i}                 &                                           \\
      \text{s.t.} \quad & s_j \geq s_i + p_i - M^1_{ij} (1 - x_{ij}), & \forall i,j \in L,\ i \ne j \\
                        & s_i + p_i \leq d_i + M^2_{i} y_i,          & \forall i \in L             \\
                        & x_{ij} + x_{ji} = 1,                 & \forall i,j \in L,\ i \ne j \\
                        & x_{ij} \in \left\{0,~1\right\},      & \forall i,j \in L                         \\
                        & y_{i} \in \left\{0,~1\right\},       & \forall i \in L                           \\
                        & s_{i} \geq r_{i},                    & \forall i \in L
    \end{align}
$$

In [18]:
from docplex.mp.model import Model

def model_for(instance: Instance) -> Model:
    model = Model("Single-Machine Scheduling")
    
    L = instance.getNbItems()
    
    # upper bound for y : temps max * L + biggest realease time
    M = max([instance.getPi(i) for i in range(L)]) * L + max([instance.getRi(i) for i in range(L)]) * 999

    # Create si, yi
    s = model.integer_var_list(L, 0, M, name="s_")
    #s = model.continuous_var_list(L, 0, M, name=f's_')
    y = model.binary_var_list(L, name="y_")
    
    # Create xi
    x = []
    for i in range(L):
        x.append(model.binary_var_list(L, name=f"x{i}_"))
        
    # Objective contraint
    model.minimize(model.sum(y[i] for i in range(L)))
    
    # --- Constraints ---------
    # sj >= si + pi - M(1, ij)(1-xij)
    for i in range(L):
        for j in range(L):
            if i != j:
                model.add_constraint(s[j] >= s[i] + instance.getItem(i)[0] - M * (1 - x[i][j]))

    # si + pi <= di + M(2, i)(yi)
    for i in range(L):
        model.add_constraint_(s[i] + instance.getPi(i) <= instance.getDi(i) + M * y[i])
      
    for i in range(L):
        # xij + xji = 1
        for j in range(i):
            model.add_constraint_(x[i][j]+x[j][i]==1)          
    for i in range(L):    
        # si >= ri    
        model.add_constraint_(s[i]>=instance.getRi(i))
        
    # test - add yi 
        
    # -------------------------
        
    return model
    

In [19]:
mdl = model_for(instance)
mdl

docplex.mp.Model['Single-Machine Scheduling']

### 3.2. Resolution

Complete the function below to construct a proper `Solution` from the obtained
`docplex` solution.

In [20]:
def solve(instance: Instance) -> Solution | None:
    model = model_for(instance)

    # enable logging
    model.log_output = True

    # solve
    solution = model.solve()
    
    if not solution:
        return None

    # TODO: convert the docplex solution to an appropriate Solution
    print("---------------")
    sol = Solution()
    sol.setInstance(instance)
    sol.setObjective(solution.get_objective_value())
    
    for i in range(instance.getNbItems()):
        print(solution.get_value(f"y__{i}"))
        sol.setSi(i, solution.get_value(f"s__{i}"))
  
    return sol

In [37]:
def test_instance(instance):
    solution_generated = solve(instance)
    print(instance.getItems())
    print("S = ", solution_generated.getS())
    print("obj = ", solution_generated.getObjective())
    print("\n")

    #pi, ri, di
    print(check_solution(solution_generated))
    print(check_solution_ontime(solution_generated))

In [38]:
test_instance(instance)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 2.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 4 rows and 4 columns.
MIP Presolve modified 8 coefficients.
Aggregator did 6 substitutions.
Reduced MIP has 16 rows, 14 columns, and 44 nonzeros.
Reduced MIP has 10 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 16 rows, 14 columns, and 44 nonzeros.
Reduced MIP has 10 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 4.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes             

In [40]:
test_instance(generate_instance(3))

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 3 rows and 3 columns.
MIP Presolve modified 11 coefficients.
Aggregator did 3 substitutions.
Reduced MIP has 9 rows, 9 columns, and 24 nonzeros.
Reduced MIP has 6 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 4 coefficients.
Reduced MIP has 9 rows, 9 columns, and 24 nonzeros.
Reduced MIP has 6 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 3.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 0.00 sec. (0.02 t

## 4. Evaluation

### 4.1. Simple evaluation

Evaluate the model and solution implemented above on various instances generated by
your method.

What size of instances can your model solve? Which parameters have the biggest impact
on the resolution time?

### 4.2. Relaxation

Evaluate the impact of having tighter big-$M$ values in your model to the appropriate
relaxation.

**Tips:**

- Adapt the `model_for` function (or create a new one) to easily change the big-$M$
  values.
- You can use the following code to solve the relaxation of your model:

```python
from docplex.mp.relax_linear import LinearRelaxer

lp = LinearRelaxer.make_relaxed_model(model)
solution_relaxed = lp.solve(log_output=True)
```

In [26]:
from docplex.mp.relax_linear import LinearRelaxer

lp = LinearRelaxer.make_relaxed_model(model)
solution_relaxed = lp.solve(log_output=True)

print("S = ", solution_relaxed.getS())
print("obj = ", solution_relaxed.getObjective())
visualization(instance, solution_relaxed)

NameError: name 'model' is not defined

### 4.3. Visualization (Bonus)

Propose a method to visualize the instance and solution obtained by your model using
library such as `matplotlib`.

In [23]:

# corriger pour permettre que les taches soient pas dans le même ordre
def visualization(instance: Instance, solution:Solution):
    current = 0
    
    sol = []
    for index, si in enumerate(solution.getS()):
        sol.append([si, index])
    sol.sort()
    
    for si, index in sol:
        #pi, ri, di
        pi = instance.getPi(index)
        wait_time = abs(current-si)
        print("-"*int(wait_time), end="")
        print((str)(index)*pi, end="")
        current = pi+si

In [25]:
print(instance.getItems())
sol = Solution()
sol.setInstance(instance)
sol.setS([2, 8, 16, 7])
visualization(instance, sol)

[[5, 2, 10], [7, 1, 15], [5, 3, 22], [1, 4, 8]]
--0000031111111-22222